In [ ]:
#http://data.portic.fr/api/ports?param=&shortenfields=false&both_to=false&date=1787
import http.client
from io import StringIO, BytesIO, TextIOWrapper
import json
import pandas as pd
from flask import Flask, render_template, request, jsonify
from math import pi, log
 

from bokeh.embed import components
from bokeh.plotting import figure
from bokeh.resources import INLINE

from bokeh.models import ColumnDataSource
#from bokeh.palettes import Spectral6
from bokeh.transform import factor_cmap
#from bokeh.palettes import d3
#from bokeh.palettes import  all_palettes
#from bokeh.palettes import Turbo256
import colorcet as cc
#from colormap import rgb2hex



app = Flask(__name__)

def getStateForDate(jsonelement, dateparam):
        """
        jsonelement : [{"1749-1815" : "Toscane"},{"1801-1807" : "Royaume d'Étrurie"},{"1808-1814" : "Empire français"}]
        Internal method to output state for 1787 as dateparam
        return name of the state of the period including dateparam 
        state = getStateForDate(json.load(StringIO('{"1749-1815" : "Toscane"}')), 1787)
        return Toscane
        """
        for dates, state in jsonelement.items():
            datesarray = dates.split('-')
            start = datesarray[0]
            end = datesarray[1]
            if (int(start) <= dateparam <= int(end)):
                return state
            
def getPorticData(local = True):
    conn = http.client.HTTPConnection("data.portic.fr")
    conn.request("GET", "/api/ports/?shortenfields=false&both_to=false&date=1787")
    r1 = conn.getresponse()
    #print(r1.status, r1.reason)
    #print(r1)

    data1 = r1.read()  # This will return entire content.
    #print(type(data1)) #bytes
    b = BytesIO(data1)
    b.seek(0) #Start of stream (the default).  pos should be  = 0;
    data = json.load(b)
    #print(type(data))

    #print(data)
    type(data)

    df = pd.DataFrame(data)

    #print(df.shape)

    df.columns

    #df.admiralty.unique()

    # Dealing with null values
    df.admiralty.isnull().values.any() #True 
    values = {'admiralty': 'X'}
    df = df.fillna(value=values)
    df.admiralty.isnull().values.any() #False 

    #df.admiralty.unique()
    #df.admiralty.unique().size #52
    
    #https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html
    for elt in df['belonging_states'].unique():
        if elt is None :
            df.loc[df.belonging_states.eq(elt), 'state_1787'] = 'X' 
        else :
            eltjson = json.loads(elt) 
            for k in eltjson :
                state = getStateForDate(k, 1787)
                if state is not None:
                    df.loc[df.belonging_states.eq(elt), 'state_1787'] = state

    ## Check the results
    #df[['toponym', 'belonging_states', 'state_1787']]        
    #type(df)
    #df.loc[0, ['toponym', 'belonging_states', 'state_1787']]

    return df

def build_viz_withparam(df, x=None):
    
    #x = 'Espagne'
    
    ## Filter on state
    if x is not None : 
        df = df.query('state_1787=="{}"'.format(x))
        
    df = df.sort_values(by=['state_1787', 'toponym'], ascending=True)
    
    # How many ports by belonging_states ?
    #print(df.groupby('state_1787')['ogc_fid'].count())
    
    # toponym will be factor, we cannot have duplicates
    df.drop_duplicates(subset=['toponym'], inplace=True)
              
    ## Init the palette to color the port according to their belonging state
    #https://stackoverflow.com/questions/3380726/converting-a-rgb-color-tuple-to-a-six-digit-code
    palette_size = len(pd.unique(df.state_1787)) #How many states ? 
    palette_state = list()
    #About colorcet cc : https://colorcet.holoviz.org/getting_started/index.html#usage
    for item in cc.glasbey_bw[:palette_size]:
        vals = [int(round(255*x)) for x in item]
        #print('#' + ''.join(['{:02X}'.format(x) for x in vals]))
        palette_state.append('#' + ''.join(['{:02X}'.format(x) for x in vals]))
    palette_state = tuple(palette_state)
  
    #Build the data source for the figure
    source = ColumnDataSource(data=dict(names=df['toponym'], counts=df['total'], country=df['state_1787']))

    # init a basic bar chart:
    # http://bokeh.pydata.org/en/latest/docs/user_guide/plotting.html#bars
    fig = figure(x_range=df['toponym'], 
                 plot_width=1200, plot_height=800, 
                 title="Counts of toponyms citation for ports per state in 1787")

    fig.vbar(x='names', #port names on x axis
             top='counts', #Number of citations in y axis
             width=0.9, 
             source=source, #data source
             legend_field="country", #legend based on the name of the state of the port (its country)
             line_color='white', 
             fill_color=factor_cmap('country', palette=palette_state, factors=pd.unique(df.state_1787))
             #color of the bar according to the country
            )
    
    
    """
    axis orientation with Bokeh,
    from : https://docs.bokeh.org/en/latest/docs/user_guide/styling.html
    """
    
    fig.xaxis.major_label_orientation = pi/2   #avec une donnée numérique de 90°
    fig.yaxis.major_label_orientation = "vertical" #ou bien, avec une orientation definie, 
    
    
    fig.xgrid.grid_line_color = None
    fig.y_range.start = 0
    fig.y_range.end = (max(df['total'])*1.05)
    #https://docs.bokeh.org/en/latest/docs/user_guide/styling.html#styling-legends
    fig.legend.orientation = "vertical"
    fig.legend.location = "top_right"
    fig.legend.label_text_font_size = "8px"
    fig.legend.spacing = 1
    fig.legend.margin = 3
    fig.legend.label_width = 20
    fig.legend.label_height = 10
    fig.legend.glyph_width = 10
    fig.legend.glyph_height = 10

    return fig

@app.route('/')
def hello_world():
    #return 'Hello, World!'
    """
    Initial plot 
    """
    df = getPorticData(local = False)
    fig = build_viz_withparam(df)#x='Espagne'

    # grab the static resources
    js_resources = INLINE.render_js()
    #print(js_resources)
    css_resources = INLINE.render_css()

    # render template
    script, div = components(fig)

    html = render_template(
        'ajax.html',
        plot_script=script,
        plot_div=div,
        js_resources=js_resources,
        css_resources=css_resources,
    )

    return html

@app.route('/show_port')
def toto():
    # Dump dataframe as html
    df = getporticdata()
    return render_template('hello.html', msg=df.to_html())



@app.route('/ajaxviz')
def formviz():
    
    full = getPorticData(local = False)
    df = full
    #df = full.iloc[0:10] #Get just 10 first lines
        
    #Parse the param
    state = request.args.get("state")
    if (state is not None and len(state)>0) :
        fig = build_viz_withparam(df, state)
    else:
        fig = build_viz_withparam(df)

    # render template
    script, div = components(fig)
      
    # pass the div and script to render_template    
    return jsonify(
        html_plot=render_template('update_figure.html', plot_script=script, plot_div=div)
    )

if __name__ == '__main__':
    app.run(port=5050) # Permet de lancer le serveur directement depuis python en exécutant le programme



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Nov/2021 21:57:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 21:57:48] "GET /ajaxviz?state=France HTTP/1.1" 200 -
127.0.0.1 - - [09/Nov/2021 21:58:02] "GET /ajaxviz?state=Monaco HTTP/1.1" 200 -
